In [1]:
import finspector
import pandas as pd
import random
import torch
import numpy as np
from transformers import BertTokenizer, BertForMaskedLM, AlbertTokenizer, AlbertForMaskedLM, RobertaTokenizer, RobertaForMaskedLM, DistilBertTokenizer, DistilBertModel, MPNetTokenizer, MPNetModel, PegasusForConditionalGeneration, PegasusTokenizer, T5ForConditionalGeneration, T5Tokenizer

In [2]:
# Prepare Three Large Language Models, BERT, RoBERTa, and ALBERT to Generate Pseudo-Log-Likelihoods with them.

model_names = ['bert', 'roberta', 'albert'] ## , 'distilbert', 'mpnet' generates errors
models_to_test = []

for model_name in model_names:

    if model_name == "bert":
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        model = BertForMaskedLM.from_pretrained('bert-base-uncased')
        uncased = True
    elif model_name == "roberta":
        tokenizer = RobertaTokenizer.from_pretrained('roberta-large')
        model = RobertaForMaskedLM.from_pretrained('roberta-large')
        uncased = False
    elif model_name == "albert":
        tokenizer = AlbertTokenizer.from_pretrained('albert-xxlarge-v2')
        model = AlbertForMaskedLM.from_pretrained('albert-xxlarge-v2')
        uncased = False
    elif model_name == 'distilbert':
        tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
        model = DistilBertModel.from_pretrained("distilbert-base-uncased")
        uncased = True
    elif model_name =='mpnet':
        tokenizer = MPNetTokenizer.from_pretrained("microsoft/mpnet-base")
        model = MPNetModel.from_pretrained("microsoft/mpnet-base")
        uncased = False
    
    model.eval()
    if torch.cuda.is_available():
        model.to('cuda')

    mask_token = tokenizer.mask_token
    log_softmax = torch.nn.LogSoftmax(dim=0)

    lm = {
        "name": model_name,
        "model": model,
        "tokenizer": tokenizer,
        "mask_token": mask_token,
        "log_softmax": log_softmax,
        "uncased": uncased
    }
    
    models_to_test.append(lm)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
# Get scores functions.
# Original Code: https://github.com/nyu-mll/crows-pairs

def get_log_prob_unigram(masked_token_ids, token_ids, mask_idx, lm):
    """
    Given a sequence of token ids, with one masked token, return the log probability of the masked token.
    """
    
    model = lm["model"]
    tokenizer = lm["tokenizer"]
    log_softmax = lm["log_softmax"]
    mask_token = lm["mask_token"]
    uncased = lm["uncased"]

    # get model hidden states
    output = model(masked_token_ids)
    hidden_states = output[0].squeeze(0)
    mask_id = tokenizer.convert_tokens_to_ids(mask_token)

    # we only need log_prob for the MASK tokens
    assert masked_token_ids[0][mask_idx] == mask_id

    hs = hidden_states[mask_idx]
    target_id = token_ids[0][mask_idx]
    log_probs = log_softmax(hs)[target_id]

    return log_probs

def get_span(seq):
    seq = [str(x) for x in seq.tolist()]
    template = [x for x in range(len(seq))]
    return template


def get_score(sent, lm, n=1):
    """
    Score each sentence by masking one word at a time.
    The score for a sentence is the sum of log probability of each word in
    the sentence.
    n = n-gram of token that is masked, if n > 1, we mask tokens with overlapping
    n-grams.
    """
    model = lm["model"]
    tokenizer = lm["tokenizer"]
    log_softmax = lm["log_softmax"]
    mask_token = lm["mask_token"]
    uncased = lm["uncased"]

    if torch.cuda.is_available():
        torch.set_default_tensor_type('torch.cuda.FloatTensor')

    if uncased:
        sent = sent.lower()

    # tokenize
    sent_token_ids = tokenizer.encode(sent, return_tensors='pt')

    # get spans of non-changing tokens
    template = get_span(sent_token_ids[0])

    N = len(template)  # num. of tokens that can be masked
    mask_id = tokenizer.convert_tokens_to_ids(mask_token)
    
    sent_log_probs = 0.
    total_masked_tokens = 0

    # skipping CLS and SEP tokens, they'll never be masked
    for i in range(1, N-1):
        sent_masked_token_ids = sent_token_ids.clone().detach()

        sent_masked_token_ids[0][template[i]] = mask_id
        total_masked_tokens += 1

        score = get_log_prob_unigram(sent_masked_token_ids, sent_token_ids, template[i], lm)

        sent_log_probs += score.item()

    lpscore = {}
    # average over iterations
    lpscore["model"] = lm['name']
    lpscore["sentence"] = sent
    lpscore["score"] = sent_log_probs/(N-2)

    return lpscore

In [5]:
# generate sentence embeddings using t-SNE

from sklearn.manifold import TSNE
df = pd.read_csv('data/sample-movie-quotes.csv', index_col=False)
df['index'] = [item for item in range(df.shape[0])]
embvec = TSNE(n_components=2, perplexity=2).fit_transform(df[['bert', 'roberta', 'albert']]).tolist()

In [6]:
# launch Finspector
lh = finspector.Visualize(df, embedding = embvec, models = models_to_test, score_fn = get_score, sent_col='sent', cat_col='bias_type', spl_col='stereo_type', 
                          model_cols=['bert', 'roberta', 'albert'], other_cols=['sent_index', 'stereo_type'])
lh

Visualize(data=[{'bias_type': 'godfather', 'sent_index': 0, 'para_index': 0, 'stereo_type': 0, 'more_or_less':…